In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('model.csv')

In [ ]:
df.head()

,City,Latitude,Longitude,Gender,Senior_Citizen,Partner,Tenure_Months,Contract,Payment_Method,Monthly_Charges,Churn_Value,Churn_Score
0,Los_Angeles,33.964131,-118.272783,Male,No,No,2,Month_to_month,Mailed_check,53.85,1,86
1,Los_Angeles,34.059281,-118.307420,Female,No,No,2,Month_to_month,Electronic_check,70.70,1,67
2,Los_Angeles,34.048013,-118.293953,Female,No,No,8,Month_to_month,Electronic_check,99.65,1,86
3,Los_Angeles,34.062125,-118.315709,Female,No,Yes,28,Month_to_month,Electronic_check,104.80,1,84
4,Los_Angeles,34.039224,-118.266293,Male,No,No,49,Month_to_month,Bank_transfer_automatic,103.70,1,89


In [ ]:
df['']

In [4]:
df.shape

(7043, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             7043 non-null   object 
 1   Latitude         7043 non-null   float64
 2   Longitude        7043 non-null   float64
 3   Gender           7043 non-null   object 
 4   Senior_Citizen   7043 non-null   object 
 5   Partner          7043 non-null   object 
 6   Tenure_Months    7043 non-null   int64  
 7   Contract         7043 non-null   object 
 8   Payment_Method   7043 non-null   object 
 9   Monthly_Charges  7043 non-null   float64
 10  Churn_Value      7043 non-null   int64  
 11  Churn_Score      7043 non-null   int64  
dtypes: float64(3), int64(3), object(6)
memory usage: 660.4+ KB


In [6]:
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import *

In [7]:
df.columns

Index(['City', 'Latitude', 'Longitude', 'Gender', 'Senior_Citizen', 'Partner',
       'Tenure_Months', 'Contract', 'Payment_Method', 'Monthly_Charges',
       'Churn_Value', 'Churn_Score'],
      dtype='object')

# Encoding

In [8]:
ct1 = ColumnTransformer(
    transformers = [
        ('ct1_City' , OrdinalEncoder(handle_unknown = 'use_encoded_value' , unknown_value = -1), ['City'])  , 
        ('ct1_Gender' , OrdinalEncoder(handle_unknown = 'use_encoded_value' , unknown_value = -1), ['Gender'])  , 
        ('ct1_Senior_Citizen' , OrdinalEncoder(handle_unknown = 'use_encoded_value' , unknown_value = -1), ['Senior_Citizen'])  , 
        ('ct1_Partner' , OrdinalEncoder(handle_unknown = 'use_encoded_value' , unknown_value = -1), ['Partner'])  , 
        ('ct1_Contract' , OrdinalEncoder(handle_unknown = 'use_encoded_value' , unknown_value = -1), ['Contract'])  , 
        ('ct1_Payment_Method' , OrdinalEncoder(handle_unknown = 'use_encoded_value' , unknown_value = -1), ['Payment_Method'])  , 
        ] ,
    remainder = 'passthrough'
)

In [9]:
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.neural_network import *
from sklearn.naive_bayes import *
from xgboost  import *
from catboost import *
from lightgbm import *

In [10]:
model = RandomForestClassifier(n_estimators = 300)

In [11]:
ct2 = model

In [12]:
from imblearn.over_sampling import SMOTE

In [13]:
sampling = SMOTE(random_state = 42)

In [14]:
ct3 = sampling

In [15]:
pipe = Pipeline([
    ('p1' , ct1) , 
    ('p3' , ct3) , 
    ('p2' , ct2) , 
])

In [16]:
from sklearn.model_selection import *

In [17]:
new_df  = df.copy()

In [18]:
x = new_df.drop('Churn_Value' , axis = 1)
y = new_df['Churn_Value']

In [19]:
x_train , x_test , y_train, y_test  = train_test_split(x, y, test_size = 0.2 , random_state = 42)

In [20]:
pipe.fit(x_train, y_train)

,steps,"[('p1', ...), ('p3', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('ct1_City', ...), ('ct1_Gender', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [21]:
y_pred = pipe.predict(x_test)

In [22]:
pipe.score(x_train , y_train)

1.0

In [23]:
accuracy_score(y_test , y_pred)

0.9176721078779276

In [24]:
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.96      0.93      0.94      1009
           1       0.83      0.89      0.86       400

    accuracy                           0.92      1409
   macro avg       0.89      0.91      0.90      1409
weighted avg       0.92      0.92      0.92      1409



In [25]:
confusion_matrix(y_test , y_pred)

array([[937,  72],
       [ 44, 356]])

In [26]:
import joblib

joblib.dump(pipe , 'churn_model.pkl')

['churn_model.pkl']

In [28]:
df['Churn_Value'].value_counts()

Churn_Value
0    5174
1    1869
Name: count, dtype: int64